In [8]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.3 MB/s eta 0:00:00


In [9]:
import numpy as np
import pandas as pd
import requests
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
# from bs4 import BeautifulSoup
# import nltk
# nltk.download('stopwords')
# from nltk.corpus import stopwords
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments,EarlyStoppingCallback
import warnings

# Set pandas display options
pd.set_option("display.max_colwidth", 200)

# Ignore warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/amazon_food_review_xlnet/Reviews.csv")

In [ ]:
data.drop_duplicates(subset=['Text'],inplace=True)#dropping duplicates
data.dropna(axis=0,inplace=True)#dropping na

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data[['Text', 'Summary']].sample(5)

In [ ]:
stop_words = set(stopwords.words('english'))

def text_cleaner(text,num):
    newString = text.lower()
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    # newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    newString = re.sub('[m]{2,}', 'mm', newString)
    if(num==0):
        tokens = [w for w in newString.split() if not w in stop_words]
    else:
        tokens=newString.split()
    long_words=[]
    for i in tokens:
        if len(i)>1:                                                 #removing short word
            long_words.append(i)
    return (" ".join(long_words)).strip()

In [ ]:
#call the function
cleaned_text = []
for t in data['Text']:
    cleaned_text.append(text_cleaner(t,0))

In [ ]:
cleaned_text[:5]

In [ ]:
cleaned_summary = []
for t in data['Summary']:
    cleaned_summary.append(text_cleaner(t,1))

In [ ]:
cleaned_summary[:10]

In [ ]:
data['cleaned_text']=cleaned_text
data['cleaned_summary']=cleaned_summary

In [ ]:
data.replace('', np.nan, inplace=True)
data.dropna(axis=0,inplace=True)

In [ ]:
import matplotlib.pyplot as plt

text_word_count = []
summary_word_count = []

# populate the lists with sentence lengths
for i in data['cleaned_text']:
      text_word_count.append(len(i.split()))

for i in data['cleaned_summary']:
      summary_word_count.append(len(i.split()))

length_df = pd.DataFrame({'text':text_word_count, 'summary':summary_word_count})

length_df.hist(bins = 30)
plt.show()

In [ ]:
cnt=0
for i in data['cleaned_summary']:
    if(len(i.split())<=8):
        cnt=cnt+1
print(cnt/len(data['cleaned_summary']))

In [ ]:
max_text_len=30
max_summary_len=8

In [ ]:
cleaned_text =np.array(data['cleaned_text'])
cleaned_summary=np.array(data['cleaned_summary'])

short_text=[]
short_summary=[]

for i in range(len(cleaned_text)):
    if(len(cleaned_summary[i].split())<=max_summary_len and len(cleaned_text[i].split())<=max_text_len):
        short_text.append(cleaned_text[i])
        short_summary.append(cleaned_summary[i])

df=pd.DataFrame({'text':short_text,'summary':short_summary})

In [ ]:
df.sample(10)

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/amazon_food_review_xlnet/preprocessed_df.csv', index=False)

In [ ]:
df.sample(10)

## Loading processed data

In [10]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/amazon_food_review_xlnet/preprocessed_df.csv')
df.sample(5)

,text,summary
41426,year product longer available stores shopped groceries delighted find amazon com lower price still available chocolate version stores,excellent sugar free cooked pudding
170879,enjoyed various flavors really nice company choice flavors fun try would order,great choice
4322,tried many different cheeses say gorgonzola favorite texture aroma flavor bold pleasant yet overbearing highly recommended cheese novice cheese enthusiasts,my new favorite cheese
116123,invited stay friends friend never met basket like arrives advance hosts know grateful hospitality fruit comes fresh attractively packed cheese nuts nice bonus,perfect hostess gift
48776,big fan sugar free torani syrups received packaged three bottles one leaking really sure get replaced great product though,tasty torani syrup


In [11]:
df.shape

(214901, 2)

In [12]:
traning_set = df.sample(30000)

In [21]:
train_df, eval_df = train_test_split(traning_set, test_size=0.2, random_state=42)

In [ ]:
hf_api = 'c951279b61000ec196651e2204f6d35ad0374e33'

In [23]:
from torch.utils.data import Dataset
import pandas as pd

class TextSummarizationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, text_max_length=30, summary_max_length=8):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.text_max_length = text_max_length
        self.summary_max_length = summary_max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Extract input text and summary
        text = self.dataframe.iloc[idx]["text"]
        summary = self.dataframe.iloc[idx]["summary"]

        # Tokenize input text
        input_encodings = self.tokenizer(
            text,
            max_length=self.text_max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        # Tokenize summary
        target_encodings = self.tokenizer(
            summary,
            max_length=self.summary_max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        # Remove batch dimension
        input_ids = input_encodings["input_ids"].squeeze(0)
        attention_mask = input_encodings["attention_mask"].squeeze(0)
        labels = target_encodings["input_ids"].squeeze(0)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

In [28]:
import pandas as pd
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# Create datasets
train_dataset = TextSummarizationDataset(train_df, tokenizer)
eval_dataset = TextSummarizationDataset(eval_df, tokenizer)

In [ ]:
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Load T5 model
model = T5ForConditionalGeneration.from_pretrained("t5-base")


# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",        # Save at the end of each epoch
    load_best_model_at_end=True,  # Load the best model at the end
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    remove_unused_columns=False,
    logging_dir='./logs',
    save_total_limit=3,
    fp16=True,
)

# Define early stopping callback
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,  # Number of epochs with no improvement after which training will stop
    early_stopping_threshold=0.0   # Minimum improvement to consider for early stopping
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Use the evaluation dataset
    tokenizer=tokenizer,
    callbacks=[early_stopping_callback]
)

In [ ]:
# Start training
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.408900,1.825549
2,1.871100,1.774555
3,1.778400,1.758647
4,1.707300,1.748062
5,1.673900,1.738571
6,1.630800,1.738182
7,1.598400,1.738673
8,1.574300,1.741555
9,1.551000,1.740933


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=6750, training_loss=1.7416179108796297, metrics={'train_runtime': 2106.2866, 'train_samples_per_second': 113.945, 'train_steps_per_second': 3.561, 'total_flos': 7707123302400000.0, 'train_loss': 1.7416179108796297, 'epoch': 9.0})

In [ ]:
# Save the fine-tuned model
torch.save(model.state_dict(), "/content/drive/MyDrive/Colab Notebooks/amazon_food_review_xlnet/base_model.pt")

print("Model state_dict saved successfully.")

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# # Load the fine-tuned model and tokenizer
model_name = '/content/drive/MyDrive/Colab Notebooks/amazon_food_review_xlnet/fine_tuned_t5'  # Path to your fine-tuned model
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Set the model to evaluation mode
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
model.eval()

def summarize_text(input_text):
    # Prepend the "summarize: " prefix
    input_text = f"summarize: {input_text}"

    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=30, truncation=True)

    # Generate the summary
    with torch.no_grad():
        summary_ids = model.generate(input_ids, max_length=8, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decode the summary ids to text
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [ ]:
# Example input text for prediction
sample = data[['Text', 'Summary']].sample()
print('Text:---', sample['Text'].values[0])
print('Actual Summary:---', sample['Summary'].values[0])
# Generate summary
summary = summarize_text(sample['Text'].values[0])
print("predicted summary:---", summary)

Text:--- Kids love it, Mom & I love it, and even the dog loves it!  Whether it's for breakfast or just munching on a couple of handfuls for a snack, Cinnamon Life satisfies the hunger crave in all of us!  The sweet cinnamon taste seems to really appeal to breakfast cereal eaters alike.  The only slight complaint I have is it gets soggy really fast, so I try not to put a lot of milk on my cereal.
Actual Summary:--- A Better Taste Than The Regular Life Cereals!
predicted summary:--- kids love it, mom &


In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from torch.utils.data import DataLoader, Dataset

# ====== Load Model and Tokenizer ======
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/amazon_food_review_xlnet/base_model.pt"  # Update this path
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer (ensure it's the same one used during training)
tokenizer = T5Tokenizer.from_pretrained("t5-small")  # Change if needed

# Load model architecture
model = T5ForConditionalGeneration.from_pretrained("t5-small")  # Change if needed

# Load trained weights
model.load_state_dict(torch.load(checkpoint_path, map_location=device))
model.to(device)
model.train()  # Set model to training mode

# ====== Define Dataset and DataLoader ======
class CustomDataset(Dataset):
    def __init__(self, texts, summaries, tokenizer, max_length=512):
        self.texts = texts
        self.summaries = summaries
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        input_text = f"summarize: {self.texts[idx]}"
        target_text = self.summaries[idx]

        # Tokenization
        inputs = self.tokenizer(input_text, padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt")
        targets = self.tokenizer(target_text, padding="max_length", truncation=True, max_length=50, return_tensors="pt")

        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": targets["input_ids"].squeeze()
        }

# Example data (Replace with your dataset)
texts = ["This is an example input text for summarization."]
summaries = ["Example summary."]

# Create dataset and dataloader
train_dataset = CustomDataset(texts, summaries, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

# ====== Define Optimizer and Loss ======
optimizer = AdamW(model.parameters(), lr=5e-5)

# ====== Training Loop ======
epochs = 3
for epoch in range(epochs):
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

In [32]:
# ====== Save Updated Model ======
torch.save(model.state_dict(), "/content/drive/MyDrive/Colab Notebooks/amazon_food_review_xlnet/summurization_model_1march2025.pt")
print("Training complete. Model saved.")

Training complete. Model saved.


In [18]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from torch.utils.data import DataLoader, Dataset

# ====== Load Model and Tokenizer ======
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/amazon_food_review_xlnet/base_model.pt"  # Update this path
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer (ensure it's the same one used during training)
tokenizer = T5Tokenizer.from_pretrained("t5-base")  # Change if needed

# Load model architecture
model = T5ForConditionalGeneration.from_pretrained("t5-base")  # Change if needed

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [19]:
state_dict = torch.load(checkpoint_path, map_location=device)  # Load to CPU first
model.load_state_dict(state_dict, strict=False)  # Allow partial loading
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [29]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",        # Save at the end of each epoch
    load_best_model_at_end=True,  # Load the best model at the end
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    remove_unused_columns=False,
    logging_dir='./logs',
    save_total_limit=3,
    fp16=True,
)

# Define early stopping callback
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,  # Number of epochs with no improvement after which training will stop
    early_stopping_threshold=0.0   # Minimum improvement to consider for early stopping
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Use the evaluation dataset
    tokenizer=tokenizer,
    callbacks=[early_stopping_callback]
)

In [30]:
# Start training
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.551300,1.698368
2,1.673200,1.678994
3,1.624900,1.681498
4,1.580800,1.681091
5,1.539300,1.693161


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=3750, training_loss=1.5926185791015626, metrics={'train_runtime': 1086.2732, 'train_samples_per_second': 220.939, 'train_steps_per_second': 6.904, 'total_flos': 4281735168000000.0, 'train_loss': 1.5926185791015626, 'epoch': 5.0})

In [27]:
import pandas as pd
from torch.utils.data import Dataset
import pandas as pd

class TextSummarizationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, text_max_length=30, summary_max_length=8):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.text_max_length = text_max_length
        self.summary_max_length = summary_max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Extract input text and summary
        text = self.dataframe.iloc[idx]["text"]
        summary = self.dataframe.iloc[idx]["summary"]

        # Ensure summary is a string
        summary = str(summary) # convert summary to string type

        # Tokenize input text
        input_encodings = self.tokenizer(
            text,
            max_length=self.text_max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        # Tokenize summary
        target_encodings = self.tokenizer(
            summary,
            max_length=self.summary_max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        # Remove batch dimension
        input_ids = input_encodings["input_ids"].squeeze(0)
        attention_mask = input_encodings["attention_mask"].squeeze(0)
        labels = target_encodings["input_ids"].squeeze(0)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

In [33]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from torch.utils.data import DataLoader, Dataset

# # ====== Load Model and Tokenizer ======
# checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/amazon_food_review_xlnet/base_model.pt"  # Update this path
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer (ensure it's the same one used during training)
tokenizer = T5Tokenizer.from_pretrained("t5-base")  # Change if needed

# Load model architecture
model = T5ForConditionalGeneration.from_pretrained("t5-base")  # Change if needed

# Set the model to evaluation mode
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
model.eval()

def summarize_text(input_text):
    # Prepend the "summarize: " prefix
    input_text = f"summarize: {input_text}"

    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=30, truncation=True)

    # Generate the summary
    with torch.no_grad():
        summary_ids = model.generate(input_ids, max_length=8, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decode the summary ids to text
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [37]:
# Example input text for prediction
sample = df[['text', 'summary']].sample()
print('Text:---', sample['text'].values[0])
print('Actual Summary:---', sample['summary'].values[0])
# Generate summary
summary = summarize_text(sample['text'].values[0])
print("predicted summary:---", summary)

Text:--- picked whim amazing sliced strawberries cracked black pepper wait douse salad next absolutely delicious
Actual Summary:--- this stuff is magic in bottle
predicted summary:--- picked whim amazing sliced
